<a href="https://colab.research.google.com/github/arsalan993/AI-based-Image-search-/blob/master/Training_Image_search.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Training Module**

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/drive/1QsyKutAkcNHphRMazKgau_heQU4-4wBM?usp=sharing)
---



## Load the Drive helper and mount



In [1]:
from google.colab import drive
from tqdm import tqdm

drive.mount('/content/drive',force_remount=True)
%cd drive/My\ Drive/Image_recommender_via_desciption

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/drive
/content/drive/My Drive/Image_recommender_via_desciption


In [2]:
!ls 

bert_lang_model  tiny-imagenet-200	Training.ipynb
model		 tiny-imagenet-200.zip


## Download and extracting tiny imagenet dataset
###### Dataset comprise of 200 classes, train folder consist of 500 images for each class while validaion folder comprises of 10000 images, 50 for each class. While test folder contains unlabbeled images.

In [3]:
# !wget http://cs231n.stanford.edu/tiny-imagenet-200.zip
# !unzip tiny-imagenet-200.zip'
# import zipfile
# zip_ref = zipfile.ZipFile("tiny-imagenet-200.zip", 'r')
# zip_ref.extractall("")
# zip_ref.close()

In [4]:
!ls tiny-imagenet-200

test  train  val  wnids.txt  words.txt


In [5]:
import pandas as pd
import os
import numpy as np

#### ***words.txt*** file consist of keys and respective labels but since it belongs to complete imagenet dataset therefore we filter keys for which we have data in tiny-imagenet*

In [6]:
categories = pd.read_csv("tiny-imagenet-200/words.txt",sep='\t',names=['key','cat'], header=None)
from glob import glob
valid_cat_index = [os.path.basename(os.path.normpath(i)) for i in glob("tiny-imagenet-200/train/*/")]
categories = categories[categories['key'].isin(valid_cat_index)]
categories['key'].nunique()

200

### We are going to install sentence transformer library. Sentence Transformer is based on BERT and it allows us to extract sentence representation of given sentence
### reference https://github.com/UKPLab/sentence-transformers

In [7]:
!pip install -U sentence-transformers
from sentence_transformers import SentenceTransformer

     |████████████████████████████████| 71kB 4.0MB/s 
     |████████████████████████████████| 778kB 12.8MB/s 
     |████████████████████████████████| 1.1MB 12.7MB/s 
     |████████████████████████████████| 3.0MB 43.6MB/s 
     |████████████████████████████████| 890kB 42.5MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-0.3.3-cp36-none-any.whl size=97299 sha256=5ae82ce828e3abbb299b90039b3c2b591336716ff27d9c7673a2027a104472f6
  Stored in directory: /root/.cache/pip/wheels/75/d6/0a/cab163b21d0597cc1580bc344487b11ad405e0d1d314725f2b
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp36-none-any.whl size=893260 sha256=7c47a5a27a79dd35ecfb42297d78a3760b5667bfb8a866fa9297572423fb6019
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sentence-transformers sacremoses


uncomment below code to download required pretrained language model and the save the extracted model files in required directory

In [8]:
# model = SentenceTransformer('bert-large-nli-mean-tokens')
# !cp -r /root/.cache/torch/sentence_transformers bert_lang_model

In [9]:
!ls bert_lang_model/sentence_transformers

public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-large-nli-mean-tokens.zip


In [10]:
lang_model = SentenceTransformer('bert_lang_model/sentence_transformers/public.ukp.informatik.tu-darmstadt.de_reimers_sentence-transformers_v0.2_bert-large-nli-mean-tokens.zip/')

In some cases were are given multiple labels which are basically represent same meaning for the object or you can say synonym of exact meanining. 
We extract embeddings of those words and take mean. Below function serves that purpose

In [11]:
def get_embeddings(cats):
  global lang_model
  cats = cats.split(",")
  cats = lang_model.encode(cats)
  if len(cats) > 0:
    return np.mean(np.array(cats), axis=0)
  else:
    return np.array(cats)


In [12]:
categories['cat_embed'] = categories['cat'].apply(get_embeddings)
categories.head(3)

,key,cat,cat_embed
7335,n01443537,"goldfish, Carassius auratus","[-0.097731255, -0.057093233, 0.41478953, -0.19..."
8328,n01629819,"European fire salamander, Salamandra salamandra","[-0.66994673, -0.09468903, 0.38685623, -0.3243..."
8394,n01641577,"bullfrog, Rana catesbeiana","[-0.35039034, 0.3813054, 0.77611536, 0.5907758..."


## Getting train data path

In [13]:
train_images_path = []
for i in tqdm(glob('tiny-imagenet-200/train/**')):
  train_images_path = train_images_path + glob(i+'/images/*.JPEG')

100%|██████████| 200/200 [19:11<00:00,  5.76s/it]


In [14]:
train_images_path = [(i.split("/")[2],i) for i in train_images_path]
train_images_path = pd.DataFrame(train_images_path,columns=['key','img_path'])
train_images_path = train_images_path.merge(categories, on='key', how='left')

In [15]:
train_images_path.head(2)

,key,img_path,cat,cat_embed
0,n03584254,tiny-imagenet-200/train/n03584254/images/n0358...,iPod,"[-0.73246765, -0.7871005, 0.25356412, 0.339683..."
1,n03584254,tiny-imagenet-200/train/n03584254/images/n0358...,iPod,"[-0.73246765, -0.7871005, 0.25356412, 0.339683..."


## Getting val data path

In [16]:
val_images_path = pd.read_csv("tiny-imagenet-200/val/val_annotations.txt",sep='\t',usecols=[0,1],names=['img_path','key'], header=None)
def add_val_path(path):
  return "tiny-imagenet-200/val/images/" + path
val_images_path['img_path'] = val_images_path['img_path'].apply(add_val_path)
val_images_path = val_images_path.merge(categories, on='key', how='left')

In [17]:
val_images_path.head(2)

,img_path,key,cat,cat_embed
0,tiny-imagenet-200/val/images/val_0.JPEG,n03444034,go-kart,"[-0.51237804, 0.03610748, -0.04462959, 0.50456..."
1,tiny-imagenet-200/val/images/val_1.JPEG,n04067472,reel,"[-0.33439007, 0.42378613, 0.50110006, 0.005709..."


Since Dataset is quite large on Google colab using GPU it takes around 10 hours to complete one epoch therefore set the below subset flag to true to get the small stratified sample for experimenting with your own custom model

In [18]:
subset = False
if subset == True:
  val_images_path = val_images_path.groupby(['key']).head(1).reset_index(drop=True)
  train_images_path = train_images_path.groupby(['key']).head(3).reset_index(drop=True)

**Importing PyTorch**

In [19]:
from __future__ import print_function, division
from natsort import natsorted
import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import numpy as np
import time
import os
import copy
from torch.utils.data import Dataset
from PIL import Image

%matplotlib inline
import time
import pylab as pl
from IPython import display

In [35]:
!pip install atpbar
from atpbar import atpbar


In [20]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

### Since currently we are using pretrained ResNet inorder to perform transfer learning therefore we set the transformation parameters in way you see below,  you can change it as per your model requirements 

In [53]:
mean_nums = [0.485, 0.456, 0.406]
std_nums = [0.229, 0.224, 0.225]

chosen_transforms = {'train': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]), 'val': transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
]),
}

Since data in dataset is not place in a standard format as required by default Dataset class in pytorch library therefore we created out own.

In [54]:
class CustomDataSet(Dataset):
    def __init__(self, main_dir, transform):
        # self.main_dir = main_dir
        self.transform = transform
        # all_imgs = os.listdir(main_dir)
        all_imgs = main_dir['img_path'].to_list()
        # self.total_imgs = natsorted(all_imgs)
        self.total_imgs = all_imgs
        self.labels = main_dir['cat_embed'].to_list()

    def __len__(self):
        return len(self.total_imgs)

    def __getitem__(self, idx):
        # img_loc = os.path.join(self.main_dir, self.total_imgs[idx])
        img_loc = self.total_imgs[idx]
        image = Image.open(img_loc).convert("RGB")
        tensor_image = self.transform(image)
        # print(self.labels[idx])
        tensor_label = torch.tensor(self.labels[idx], dtype=torch.float, device=device)
        # print(tensor_label)
        # print("tensor shape is ",tensor_label.shape)
        return [tensor_image,tensor_label]

In [55]:
chosen_datasets = {x[0]: CustomDataSet(x[1],chosen_transforms[x[0]])
                      for x in [('train',train_images_path), ('val',val_images_path)]}

In [56]:
dataset_sizes = {x: len(chosen_datasets[x]) for x in ['train', 'val']}


In [57]:
dataloaders = {x: torch.utils.data.DataLoader(chosen_datasets[x], batch_size=400,
                                              shuffle=True)
              for x in ['train', 'val']}

Downloading ResNet34 pretrained model and setting target layer to 1024 dimension since output of S-BERT is 1024 dimensional vector

In [ ]:
res_mod = models.resnet34(pretrained=True)
# res_mod = models.resnet152(pretrained=True)

num_ftrs = res_mod.fc.in_features
res_mod.fc = nn.Linear(num_ftrs, 1024)

Downloading: "https://download.pytorch.org/models/resnet34-333f7ec4.pth" to /root/.cache/torch/hub/checkpoints/resnet34-333f7ec4.pth


Try experimenting with different optimization function and setting up different learning rate

In [ ]:
res_mod = res_mod.to(device)
criterion = torch.nn.MSELoss() 

# Observe that all parameters are being optimized
# optimizer_ft = optim.SGD(res_mod.parameters(), lr=0.001, momentum=0.9)
optimizer_ft = optim.AdamW(res_mod.parameters(),lr=0.0015)


# Decay LR by a factor of 0.1 every 7 epochs
exp_lr_scheduler = lr_scheduler.StepLR(optimizer_ft, step_size=1, gamma=0.1)

### Performing Transfer learning
Due to large dataset target of training for 3-4 epoch and observing loss wasnt achieved
Try switching from Google Colab to paid or your personal GPU enviroment for speeding up process and completing all epochs 

In [ ]:
model = res_mod
criterion = criterion
optimizer = optimizer_ft
scheduler = exp_lr_scheduler
num_epochs=3
# def train_model(model, criterion, optimizer, scheduler, num_epochs=10):
since = time.time()

best_model_wts = copy.deepcopy(model.state_dict())
best_loss = 0.0


for epoch in atpbar(range(num_epochs),name = "Epoch status"):
    time.sleep(1)
    # print('Epoch {}/{}'.format(epoch, num_epochs - 1))
    # print('-' * 10)

    # Each epoch has a training and validation phase
    for phase in ['train', 'val']:
        time.sleep(1)
        if phase == 'train':
            scheduler.step()
            model.train()  # Set model to training mode
        else:
            model.eval()   # Set model to evaluate mode

        current_loss = 0.0
        current_corrects = 0

        # Here's where the training happens
        # print('Iterating through data...')
        # counter_ = 0
        # loss_graph = []
        for inputs, labels in atpbar(dataloaders[phase],name = phase + " for epoch "+str(epoch)):
            inputs = inputs.to(device)
            labels = labels.to(device)

            #  We need to zero the gradients, don't forget it
            optimizer.zero_grad()

            # Time to carry out the forward training poss
            # We only need to log the loss stats if we are in training phase
            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                # _, preds = torch.max(outputs, 1)
                loss = criterion(torch.flatten(outputs), torch.flatten(labels))

                # backward + optimize only if in training phase
                if phase == 'train':
                    # if counter_ % 1 == 0:
                      # print(loss)
                      # loss_graph.append(loss)
                      # pl.plot(loss_graph)
                      # display.clear_output(wait=True)
                      # display.display(pl.gcf())
                    # counter_ += 1

                    loss.backward()
                    optimizer.step()

            # We want variables to hold the loss statistics
            current_loss += loss.item() * inputs.size(0)
            # current_corrects += torch.sum(preds == labels.data)

        epoch_loss = current_loss / dataset_sizes[phase]
        # epoch_acc = current_corrects.double() / dataset_sizes[phase]

        # print('{} Loss: {:.4f} '.format(
        #     phase, epoch_loss))

        # Make a copy of the model if the accuracy on the validation set has improved
        if phase == 'val' and epoch_loss > best_loss:
            best_loss = epoch_loss
            best_model_wts = copy.deepcopy(model.state_dict())
            torch.save(model,"model/model_resnet.pth")

    print()

time_since = time.time() - since
print('Training complete in {:.0f}m {:.0f}s'.format(
    time_since // 60, time_since % 60))
print('Best val Loss: {:4f}'.format(best_loss))

# Now we'll load in the best model weights and return it
model.load_state_dict(best_model_wts)
    # return model


VBox()

/usr/local/lib/python3.6/dist-packages/torch/optim/lr_scheduler.py:123: UserWarning: Detected call of `lr_scheduler.step()` before `optimizer.step()`. In PyTorch 1.1.0 and later, you should call them in the opposite order: `optimizer.step()` before `lr_scheduler.step()`.  Failure to do this will result in PyTorch skipping the first value of the learning rate schedule. See more details at https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate
  "https://pytorch.org/docs/stable/optim.html#how-to-adjust-learning-rate", UserWarning)


Since mounted google drive never copy data directory from google colab instantly therefore syncing takes quite sometime. Below command keeps running until drive is completely synced 

In [ ]:
drive.flush_and_unmount()

### Testing model on Sample image

In [26]:
!ls model/

model_resnet.pth


In [59]:
model = torch.load("model/model_resnet.pth")
best_model_wts = copy.deepcopy(model.state_dict())
model.load_state_dict(best_model_wts)
model = model.eval()

In [60]:
pred = model(chosen_datasets['val'].__getitem__(0)[0].unsqueeze(0).to(device))

In [61]:
lab = chosen_datasets['val'].__getitem__(0)[1].unsqueeze(0).to(device)

In [62]:
cos = nn.CosineSimilarity(dim=1, eps=1e-6)

In [63]:
cos(lab, pred) 

tensor([0.6395], device='cuda:0', grad_fn=<DivBackward0>)

## Due to generalization cosine angle is not zero definitely there will be some information loss. Here are some suggestions to reduce information loss:


1.   Try different Lang model for sentence encoding like ULMFIT, USE or other.
2.   Play with the hyperparameter tuning and train it for more epoch and observe loss value.
3.   Try unfreezing the last layer of model and also add one more hidden layer to it to getter better results. This will surely improve results to quite extent.
4.   Use High resolution image atleast 256 dim, tiny imagenet constitue of image of 64x64




# **Creating Annoy indexing**
For details go to https://github.com/spotify/annoy

---



In [70]:
!pip install annoy

     |████████████████████████████████| 645kB 7.2MB/s 
  Created wheel for annoy: filename=annoy-1.16.3-cp36-cp36m-linux_x86_64.whl size=297329 sha256=5148eff107a6b27d59d342a60aa9346aa43fb71d577ef090f1b04d3b43acc29b
  Stored in directory: /root/.cache/pip/wheels/f3/01/54/6ef760fe9f9fc6ba8c19cebbe6358212b5f3b5b0195c0b813f
Successfully built annoy


In [64]:
model = torch.load("model/model_resnet.pth")
best_model_wts = copy.deepcopy(model.state_dict())
model.load_state_dict(best_model_wts)
model = model.eval()

For demo purpose and time limitation we will only index validation data.

In [80]:
from annoy import AnnoyIndex
t = AnnoyIndex(1024, 'angular') # 1024 is dimension of sentence encoding
count = 0
for inputs, _ in tqdm(chosen_datasets['val'], position=0, leave=True):
  pred = model(inputs.unsqueeze(0).to(device)).tolist()[0]
  t.add_item(count, pred)
  count += 1

t.build(100) # 10 trees
t.save('model/encoding.ann')

100%|██████████| 10000/10000 [1:13:43<00:00,  1.97it/s]





100%|██████████| 10000/10000 [1:13:43<00:00,  2.26it/s]


True

In [86]:
val_images_path[['img_path','cat']].to_csv("model/val_path.csv",index=False)